In [ ]:
# Importation of libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay

from useful_methods.plots import plot_delaunay

In [ ]:
# Importing database
df = pd.read_csv("../../database/data.csv", sep=";", decimal=",")
df.head()

In [ ]:
department = "Gard"

In [ ]:
df_gard = df.loc[df['nom_dep'] == department]
df_gard = df_gard.drop(columns=["code_op", "id_site_partage", "id_station_anfr", "nom_reg", "nom_dep", "insee_dep", "insee_com", "site_2g", "site_3g", "site_4g", "site_5g", "mes_4g_trim", "site_ZB", "site_DCC", "site_strategique", "site_capa_240mbps", "date_ouverturecommerciale_5g", "site_5g_700_m_hz", "site_5g_800_m_hz", "site_5g_1800_m_hz", "site_5g_2100_m_hz", "site_5g_3500_m_hz"])
df_gard.head()

In [ ]:
# Selecting the provider : Free Mobile
df_gard_free = df_gard.loc[df['nom_op'] == "Free Mobile"]
df_gard_free = df_gard_free.drop(columns=['nom_op'])

# Creation of points couples for Delaunay
df_gard_free_points = np.array(df_gard_free[['longitude', 'latitude']])
# Delaunay triangulation
delaunay_gard_free = Delaunay(df_gard_free_points)

In [ ]:
chemin = '../../local/travail_anneePrec/departements-20180101.shx'
geo_df_shx = gpd.read_file(chemin)
geo_df = geo_df_shx.drop([ 'wikipedia', 'nuts3', 'surf_km2'], axis = 1)
geo_df = geo_df.drop(geo_df[(geo_df['code_insee'] == '973') | (geo_df['code_insee'] == '972') | (geo_df['code_insee'] == '974') | (geo_df['code_insee'] == '976') | (geo_df['code_insee'] == '971')].index)
geo_df.index = geo_df.index.astype(str)

# On joint la Métropole de Lyon et le Rhône
from shapely.geometry import Polygon, MultiPolygon


# Coordonnées des polygones
polygone_rhone = geo_df[geo_df['code_insee'] == "69D"]['geometry'].values[0]
polygone_lyon = geo_df[geo_df['code_insee'] == "69M"]['geometry'].values[0]


# Créer un MultiPolygon avec les deux polygones
multipolygone_69 = polygone_rhone.union(polygone_lyon)

# Créer une nouvelle ligne avec code_insee = '69' et le MultiPolygon
dep_rhone = {
    'code_insee': '69',
    'nom': 'Rhône',
    'geometry': multipolygone_69
}
# Convertir la nouvelle ligne en GeoDataFrame
nouveau_gdf = gpd.GeoDataFrame([dep_rhone], geometry='geometry')

# Supprimer les anciennes lignes avec code_insee = '69'
geo_df = geo_df[geo_df['code_insee'] != '69D']
geo_df = geo_df[geo_df['code_insee'] != '69M']

# Ajouter la nouvelle ligne au GeoDataFrame
geo_df = gpd.GeoDataFrame(pd.concat([geo_df, nouveau_gdf], ignore_index=True), crs=geo_df.crs)

geo_df = geo_df.drop_duplicates()

# Suppression des zéros à gauche dans les numéros d'index
geo_df['code_insee'] = geo_df['code_insee'].str.lstrip('0')

In [ ]:
geo_df

In [ ]:
geo_df.loc[geo_df['nom']=="Gard",:].plot(edgecolor = "black", linewidth = 1, color="white")
plot_delaunay(delaunay_gard_free)